<a href="https://colab.research.google.com/github/pjpeters13/neural-network-challenge-2/blob/main/attrition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1: Preprocessing

In [101]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [102]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [103]:
# Create y_df with the Attrition and Department columns

y_df = attrition_df[['Attrition', 'Department']]


In [104]:
# Create a list of at least 10 column names to use as X data
columns_x = ["Education" , "Age",  "DistanceFromHome", "JobSatisfaction", "OverTime", "StockOptionLevel", "WorkLifeBalance", "YearsAtCompany", "YearsSinceLastPromotion", "NumCompaniesWorked"]


# Create X_df using your selected columns
X_df = attrition_df[columns_x]

# Show the data types for X_df
X_df.dtypes


Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [105]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=1)




In [121]:
# Create a LabelEncoder object
over_time_le = LabelEncoder()

# Transform the 'OverTime' column in the training data
X_train['OverTime'] = over_time_le.fit_transform(X_train['OverTime'].values.ravel())

# Transform the 'OverTime' column in the testing data
X_test['OverTime'] = over_time_le.transform(X_test['OverTime'].values.ravel())

# Optionally, display the count of each category in the training data
X_train['OverTime'].value_counts()


OverTime
0    798
1    304
Name: count, dtype: int64

In [122]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [127]:
# Create a OneHotEncoder for the Department column
department_ohe = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
department_ohe.fit(y_train[['Department']])


# Create two new variables by applying the encoder
# to the training and testing data
y_train_department_encoded = department_ohe.transform(y_train[['Department']])
y_test_department_encoded = department_ohe.transform(y_test[['Department']])



In [129]:
# Create a OneHotEncoder for the Attrition column
attrition_ohe = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_ohe.fit(y_train[['Attrition']])


# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition_encoded = attrition_ohe.transform(y_train[['Attrition']])
y_test_attrition_encoded = attrition_ohe.transform(y_test[['Attrition']])




## Create, Compile, and Train the Model

In [131]:
# Find the number of columns in the X training data
n_cols = X_train.shape[1]


# Create the input layer
input_layer = layers.Input(shape=(n_cols,), name = "input_layer")


# Create at least two shared layers
shared_layer_1 = layers.Dense(64,activation="relu",name="shared_layer_1")(input_layer)
shared_layer_2 = layers.Dense(32,activation="relu",name="shared_layer_2")(shared_layer_1)


In [111]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer


# Create the output layer



In [112]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer


# Create the output layer



In [113]:
# Create the model


# Compile the model


# Summarize the model


In [114]:
# Train the model



In [115]:
# Evaluate the model with the testing data


In [116]:
# Print the accuracy for both department and attrition


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1.
2.
3.